# Setup

In [1]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib
%matplotlib inline
matplotlib.style.use('default')
import requests
import json

In [2]:
# data and google api key setup
#google_api_access_key = open('../.env/.google_api_access_key').read()
google_api_access_key = open('../.env/.google_api_access_key_chucky').read() # chucky's key

In [3]:
# let's use our great df, we created in Grid Generator notebook
df_grid_points_final = pd.read_excel('../raw_data/raw_data_geneva_grid_points.xlsx', usecols='B:D') 
df_grid_points_final

,lat_long,lat,long
0,"46.20151452321782,6.111010044545912",46.201515,6.111010
1,"46.20213626535293,6.111010044545912",46.202136,6.111010
2,"46.20337972851531,6.111010044545912",46.203380,6.111010
3,"46.20400144954259,6.111010044545912",46.204001,6.111010
4,"46.20462316353395,6.111010044545912",46.204623,6.111010
...,...,...,...
3240,"46.1990274843178,6.1756887450025175",46.199027,6.175689
3241,"46.19778392265196,6.176587060286637",46.197784,6.176587
3242,"46.19840570700288,6.176587060286637",46.198406,6.176587
3243,"46.1990274843178,6.176587060286637",46.199027,6.176587


In [4]:
# convert grid points to numpy array for an efficient iteration which is far more efficient than iterrows()
grid_points_array = df_grid_points_final[['lat','long']].to_numpy()
print(grid_points_array.shape)
grid_points_array

(3245, 2)


array([[46.20151452,  6.11101004],
       [46.20213627,  6.11101004],
       [46.20337973,  6.11101004],
       ...,
       [46.19840571,  6.17658706],
       [46.19902748,  6.17658706],
       [46.19902748,  6.17748538]])

# Google Places Nearby Search

In [44]:
# Sample coordinate
lat, lng = 46.204215, 6.1449007
print(lat, lng)

# Sample response / Google nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'location':f'{lat},{lng}',
    'type':'restaurant',
    'radius':100,
    'region':'ch',
    'key':google_api_access_key
}

response_sample = requests.get(base_url, params=params).json()
df_sample = pd.json_normalize(response_sample['results'])\
        [['place_id',
          'name',
          'business_status',
          'price_level',
          'rating',
          'user_ratings_total',
          'geometry.location.lat',
          'geometry.location.lng']]
df_sample

46.204215 6.1449007


,place_id,name,business_status,price_level,rating,user_ratings_total,geometry.location.lat,geometry.location.lng
0,ChIJkcHaAClljEcRnswQD7fC1R8,Arthur's rivegauche,OPERATIONAL,3.0,3.8,799.0,46.204600,6.144372
1,ChIJ603rISlljEcRMl8bLqgznCw,Coop Restaurant Genève Fusterie,OPERATIONAL,2.0,3.9,522.0,46.204165,6.145162
2,ChIJk01VISlljEcR8NzSJRUyT_k,Swiss Innovative Architecture SA,OPERATIONAL,NaN,NaN,NaN,46.204069,6.145346
3,ChIJ39yIXR5wjEcRDV5EQoCAx6o,Coop Take it Genève City Fusterie,OPERATIONAL,NaN,4.7,3.0,46.204165,6.145162
4,ChIJ8yFFGSlljEcRm_RqW1MdE9M,Chez Philippe - Bar Grill,OPERATIONAL,3.0,4.3,1509.0,46.204117,6.144531
5,ChIJpSAtailljEcRV-WskLeIpv4,Brasserie Lipp,OPERATIONAL,3.0,4.4,1647.0,46.203362,6.145028
6,ChIJ-cPY2i5ljEcRu1Q3a3uMxqU,Le Baroque Restaurant Geneve,OPERATIONAL,4.0,4.1,371.0,46.204335,6.144487
7,ChIJH4t0yyN7jEcRNHVWHHiSzzo,A. Pougnier Confédération 5,OPERATIONAL,NaN,4.2,145.0,46.203914,6.144318
8,ChIJkcHaAClljEcRh-fsvGk1dNg,Leaderscafes Sa,OPERATIONAL,NaN,NaN,NaN,46.204630,6.144410
9,ChIJKb2xJilljEcRNoiNyyqTE3U,Caviar House & Prunier | Boutique & Restaurant...,OPERATIONAL,4.0,4.9,20.0,46.204383,6.145758


In [45]:
%%time

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?' # base url

df_restaurants = pd.DataFrame()

for row in grid_points_array:
    lat, lng = row[0], row[1]
    radius = 100 #meters

    params = {
        'location':f'{lat},{lng}',
        'types':'restaurant',
        'radius':radius,
        'region':'ch',
        'key':google_api_access_key
        }
    
    try:
        response_basic = requests.get(base_url, params=params).json()

        # create a temporarily dataframe to hold response
        df_temp = pd.json_normalize(response_basic['results'])\
            [['place_id',
              'name',
              'business_status',
              'types',
              'price_level',
              'rating',
              'user_ratings_total',
              'geometry.location.lat',
              'geometry.location.lng']]

        # concat and remove duplicates (beware drop_duplicates do not work due to lists values)
        drop_ref = df_temp.drop(columns=['types']).columns.to_list()
        df_restaurants = pd.concat([df_restaurants, df_temp], axis=0)\
            .drop_duplicates(drop_ref)\
            .reset_index(drop=True)
        
    except:
        continue
print(f'last_row: {row}')

last_row: [46.19902748  6.17748538]
CPU times: user 1min 16s, sys: 6.42 s, total: 1min 23s
Wall time: 17min 35s


In [46]:
df_restaurants

,place_id,name,business_status,types,price_level,rating,user_ratings_total,geometry.location.lat,geometry.location.lng
0,ChIJd6kh67ZkjEcRC1IQourQOyo,Restaurant La Tentation,OPERATIONAL,"[restaurant, point_of_interest, food, establis...",2.0,4.2,158.0,46.205232,6.114945
1,ChIJX68uubZkjEcRKKxpZ7OrgqU,Café Restaurant Palma,OPERATIONAL,"[restaurant, point_of_interest, food, establis...",NaN,3.7,141.0,46.204931,6.114021
2,ChIJPcYsG_BljEcR4kTFqBJJoK4,Le Versailles,OPERATIONAL,"[restaurant, point_of_interest, food, establis...",NaN,3.9,56.0,46.205752,6.114763
3,ChIJwzjJwrZkjEcRibYjr8ep3eY,Adat Sa,OPERATIONAL,"[restaurant, point_of_interest, food, establis...",NaN,NaN,NaN,46.205752,6.114763
4,ChIJhb4e-Ep7jEcRY38kziIGZmE,Restaurant Divan,OPERATIONAL,"[restaurant, food, point_of_interest, establis...",2.0,4.5,113.0,46.196924,6.115081
...,...,...,...,...,...,...,...,...,...
2015,ChIJuXqX8k1ljEcRjTPs8eqZ6Tk,Mr. Agron Ahmeti,CLOSED_TEMPORARILY,"[restaurant, food, point_of_interest, establis...",NaN,NaN,NaN,46.199619,6.173023
2016,ChIJN-0qEVJljEcR34z73PN9zUU,"La courbe du goût, café restaurant de Grange-C...",OPERATIONAL,"[restaurant, food, point_of_interest, establis...",2.0,4.3,250.0,46.199773,6.174561
2017,ChIJr8wyS6BljEcRc6tPa0MXzt8,MAMS,OPERATIONAL,"[restaurant, cafe, bakery, food, store, point_...",NaN,4.8,15.0,46.199695,6.175007
2018,ChIJLw7z81FljEcRGshe0-9WAB4,Bi Brasserie Italienne Sa,CLOSED_TEMPORARILY,"[restaurant, food, point_of_interest, establis...",NaN,NaN,NaN,46.201055,6.175114


In [47]:
df_restaurants.to_csv('../raw_data/raw_data_google_api_new.csv', encoding='utf-8-sig')

# Google Places Details Search

In [72]:
# Sample response / Google Places Details search
base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'

place_id = 'ChIJX68uubZkjEcRKKxpZ7OrgqU'


params = {
    'place_id':place_id,
    'key':google_api_access_key,
    'fields':'formatted_address,international_phone_number,website' #this can be changed later based on the need
}

response_details_sample = requests.get(base_url, params=params).json()
df_details_sample = pd.json_normalize(response_details_sample)
df_details_sample

,html_attributions,status,result.formatted_address,result.international_phone_number,result.website
0,[],OK,"Av. d'Aïre 93bis, 1203 Genève, Switzerland",+41 22 345 84 48,http://palmarestaurant.ch/


In [50]:
response_details_sample

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '93bis',
    'short_name': '93bis',
    'types': ['street_number']},
   {'long_name': "Avenue d'Aïre",
    'short_name': "Av. d'Aïre",
    'types': ['route']},
   {'long_name': 'Genève',
    'short_name': 'Genève',
    'types': ['locality', 'political']},
   {'long_name': 'Genève',
    'short_name': 'Genève',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Genève',
    'short_name': 'GE',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Switzerland',
    'short_name': 'CH',
    'types': ['country', 'political']},
   {'long_name': '1203', 'short_name': '1203', 'types': ['postal_code']}],
  'adr_address': '<span class="street-address">Av. d&#39;Aïre 93bis</span>, <span class="postal-code">1203</span> <span class="locality">Genève</span>, <span class="country-name">Switzerland</span>',
  'business_status': 'OPERATIONAL',
  'formatted_address': "Av. d'Aïre 9

In [53]:
df_new_places_ids = pd.read_csv('../raw_data/new_place_ids.csv') 
df_new_places_ids

,place_id
0,ChIJPcYsG_BljEcR4kTFqBJJoK4
1,ChIJwzjJwrZkjEcRibYjr8ep3eY
2,ChIJoxrpttF7jEcRkUB1EIDg3w4
3,ChIJVVXlhUp7jEcRVshpRnB30Ok
4,ChIJI05JTr5kjEcRTffOWTQzCNE
...,...
971,ChIJhZApiktljEcRO4PZOlGoR_Y
972,ChIJ9wLL6ZNljEcRNjmWVuGVqWw
973,ChIJhxG2srN6jEcRUls_UY7b5Io
974,ChIJuXqX8k1ljEcRjTPs8eqZ6Tk


In [73]:
list_new_places_ids = df_new_places_ids['place_id'].to_list()
list_new_places_ids

['ChIJPcYsG_BljEcR4kTFqBJJoK4',
 'ChIJwzjJwrZkjEcRibYjr8ep3eY',
 'ChIJoxrpttF7jEcRkUB1EIDg3w4',
 'ChIJVVXlhUp7jEcRVshpRnB30Ok',
 'ChIJI05JTr5kjEcRTffOWTQzCNE',
 'ChIJ4fJXrpVljEcRyaapN3U9k7c',
 'ChIJCyhiu8BkjEcRVKBoD1NolOE',
 'ChIJVVXFi8hkjEcRDIQY_egYzCY',
 'ChIJo4Xwi8hkjEcRIhTrMrsxCJs',
 'ChIJF1L7wJlljEcRkJ_MywelYs8',
 'ChIJM5Nyz5JkjEcRYAJCHLuq0Wc',
 'ChIJfarxIcZkjEcRPvEBKab9Y64',
 'ChIJkQ0KlchkjEcR28NiPv9l0f8',
 'ChIJC_DLKcdkjEcRPZOoI-II5jE',
 'ChIJW9uAwupkjEcRU9--Gy_zsKk',
 'ChIJ1aZf7YVljEcR_yETVs4Jpeo',
 'ChIJdRFop8hkjEcROxcUDaen8oY',
 'ChIJdRFop8hkjEcR39vwbYVIVE0',
 'ChIJKQhRA8ZkjEcR-b6uFnMRl_0',
 'ChIJv5FryexkjEcR2f9Cbmww1A0',
 'ChIJT1pzxM5kjEcR5dZxc_aq37Y',
 'ChIJAQAwrMZkjEcR6INHa_ywWhg',
 'ChIJ69dLrMZkjEcR2xzZSDWkcf0',
 'ChIJ0b6SasFkjEcR1c8GfLmYMcY',
 'ChIJ8eqe7M5kjEcR1j5Hqfb3rno',
 'ChIJE_GRU8FkjEcRvmGQMcgjZpg',
 'ChIJWeF-pMNkjEcRCKE9maeYDqg',
 'ChIJ2Tq2tWpljEcRyPziLK-5wS8',
 'ChIJrayc9sNkjEcRjKKFGqtOJZk',
 'ChIJ0duEPExljEcR03uA_t54Auw',
 'ChIJN20pWcRkjEcRUPKeeFzYcqc',
 'ChIJhy

In [106]:
%%time

base_url = 'https://maps.googleapis.com/maps/api/place/details/json?' # base url

df_restaurants_detail = pd.DataFrame()

for idx in list_new_places_ids:

    params = {
        'place_id':idx,
        'key':google_api_access_key,
        'fields':'name,formatted_address,international_phone_number,website' #this can be changed later based on the need
    }

    try:
        response_detail = requests.get(base_url, params=params).json()
        df_restaurants_detail = pd.concat([df_restaurants_detail, pd.json_normalize(response_detail)], axis=0)
        
    except:
        continue
print(f'last_row: {row}')

last_row: [46.19902748  6.17748538]
CPU times: user 20.7 s, sys: 1.56 s, total: 22.2 s
Wall time: 3min 28s


In [111]:
df_restaurants_detail['place_id'] = list_new_places_ids
df_restaurants_detail

,html_attributions,status,result.formatted_address,result.international_phone_number,result.name,result.website,place_id
0,[],OK,"Chem. des Sports 2, 1203 Genève, Switzerland",+41 22 344 49 41,Le Versailles,https://yellow.local.ch/d/iWjaP0dHU9GYe4GDduNgnw,ChIJPcYsG_BljEcR4kTFqBJJoK4
0,[],OK,"Chem. des Sports 2, 1203 Genève, Switzerland",NaN,Adat Sa,NaN,ChIJwzjJwrZkjEcRibYjr8ep3eY
0,[],OK,"Av. du Cimetière 2, 1213 Petit-Lancy, Switzerland",+41 22 792 46 39,Café Simeoni Fleurs - Espace Gourmand,NaN,ChIJoxrpttF7jEcRkUB1EIDg3w4
0,[],OK,"Av. du Cimetière 2, 1213 Jonction, Switzerland",NaN,Taverne Du Valais,NaN,ChIJVVXlhUp7jEcRVshpRnB30Ok
0,[],OK,"Av. Edmond-Vaucher 55, 1209 Genève, Switzerland",+41 22 777 77 85,Formule 10,NaN,ChIJI05JTr5kjEcRTffOWTQzCNE
...,...,...,...,...,...,...,...
0,[],OK,"Rte de Chêne 30, 1208 Genève, Switzerland",NaN,Mr. Jacques Iffland,NaN,ChIJhZApiktljEcRO4PZOlGoR_Y
0,[],OK,"Rte de Frontenex 70, 1208 Genève, Switzerland",+41 78 704 58 95,Les Pierogi - Polish Restaurant,http://lespierogi.ch/,ChIJ9wLL6ZNljEcRNjmWVuGVqWw
0,[],OK,"Chem. de la Florence 9, 1208 Genève, Switzerland",+41 22 346 36 42,Le B d'Armand,NaN,ChIJhxG2srN6jEcRUls_UY7b5Io
0,[],OK,"Rte de Chêne 64, 1207 Genève, Switzerland",+41 22 735 99 27,Mr. Agron Ahmeti,NaN,ChIJuXqX8k1ljEcRjTPs8eqZ6Tk


In [112]:
df_restaurants_detail.to_csv('../raw_data/new_place_ids_google_api.csv', encoding='utf-8-sig')